## Update rule and game designing in infinite-horizon setting

In this notebook, we benchmark the methodology for a infinite-horizon risk-sensitive inverse reinforcement learning (RL) problem by inspecting if the Gibbs measure converges to the dirac delta function. Furthermore, we design three different ways of choosing the transition probability matrix, more specifically the choice of the game $G_{n+1}$ at every round $n$. In this notebook:

- Every computation of the risk-aversion measure is done with the analytical expression
- We consider risk-aversion preferences characterized by a linear combination of the expectation and CVaR
- The cost function can be either known or (partially) unknown

**NB: This notebook will require additional coding time to put everything in a single 'learner' class instead of separate functions... We may also rename variables according to the notation we use in the preprint...**

**NB: Before publicly sharing this notebook, we must adjust the notation to be consistent with the paper.**

### Notation and hyperparameters

We use the following notation:
- A finite set of states $\mathbb{X}$ with $\text{n_states} := |\mathbb{X}| > 0$
- A finite set of actions $\mathbb{A}$ with $\text{n_actions} := |\mathbb{A}| > 0$
- A discount factor $r \in [0,1)$
- A finite set of transition probabilities, denoted by $G$, with $\text{n_games} := |G| > 0$.
- Finite sets of risk-aversion parameters $\mathcal{\kappa}$ and $\mathcal{\gamma}$ with $\kappa \in (0,1]$, $\gamma \in [0,1)$
- Whether a known cost function $C_{0}: \mathcal{X} \rightarrow \mathbb{R}$, or a partially unknown cost function $C_{\ell}: \mathcal{X} \rightarrow \mathbb{R}$, where we only know the values of the cost function
- A finite set of critera $\Xi$, denoted $\{\Xi_{\ell}\}_{\ell=1,\ldots,L}$ consisting of the tuples $\{\kappa_{\ell}, \gamma_{\ell}, r_{\ell}, C_{0}\}_{\ell}$ (if cost is known) or $\{\kappa_{\ell}, \gamma_{\ell}, r_{\ell}, C_{\ell}\}_{\ell}$ (if cost is partially unknown)

In [ ]:
# load Python libraries
import numpy as np
import matplotlib.pyplot as plt
import itertools
from time import time

In [ ]:
# initialize the dimensions of each space
n_states = 3
n_actions = 2

In [ ]:
# validate parameters
assert isinstance(n_states, int) & (n_states >= 0), "n_states must be a positive integer."
assert isinstance(n_actions, int) & (n_actions >= 0), "n_actions must be a positive integer."
print('*** All parameters are validated ***')

### Definition of functions

Let $\delta_{x}$ be the Dirac measure on $x$, and suppose $\gamma \in [0,1)$, $\kappa \in (0,1]$. For simplicity, here, all risk measures we consider are of the following form:
$$
\rho_{\mu}(X) = \int_{0}^{1} \text{CVaR}_{1-\alpha}(X) \mu( \textrm{d} \alpha),
$$
where
$$
\mu = \gamma \delta_{0} + (1-\gamma) \delta_{\kappa}.
$$
This characterizes a trade-off between risk-aversion behaviors from the CVaR at level $\kappa$, and risk-seeking behaviors from the (risk-neutral) expectation. Naturally, this notebook may be easily extended to risk measures defined as linear combinations of CVaRs at different thresholds. The function `get_analytical_risk` returns the risk measure of the cost random variable obtained for a given game.

In [ ]:
def get_analytical_risk(upsilon, action, trans_probs, costs):
    # sort costs in decreasing order
    sorted_costs = costs[np.argsort(costs[::-1])]
    sorted_pis = trans_probs[action,np.argsort(costs[::-1])]
    
    # compute CDF
    revd_cpis = np.cumsum(sorted_pis[::-1])
    
    kappa_cond = int(np.sum(upsilon[0] <= revd_cpis))
    
    # compute risk from expectation contribution
    risk_E = np.sum(sorted_pis*sorted_costs)
    
    # compute risk from CVaR contribution
    partial_risk_CVaR = np.array([sorted_costs[i]*(revd_cpis[-(i+1)] - revd_cpis[-(i+2)]) for i in range(kappa_cond-1)])
    risk_CVaR = (1/(1-upsilon[0]))*( \
                np.sum(partial_risk_CVaR) + sorted_costs[kappa_cond-1] * (revd_cpis[-(kappa_cond)] - upsilon[0]))
    
    # get risk of the linear combination
    risk = upsilon[1]*risk_E + (1-upsilon[1])*risk_CVaR

    return risk

Define the "value function" as
\begin{equation*}
    V^{*}_{G,\ell}(x) = C_{\ell}(x) + r_{\ell} \min_{a \in \mathbb{A}} \rho_{\mu_{\ell}}\Big(V^{*}_{G,\ell}(X^{x,a}_{G})\Big).
\end{equation*}
We find the optimal $V^{*}$ using fixed-point iteration, i.e. a value iteration algorithm.

In [ ]:
def compute_Vstar(upsilons, games, progress=False):
    # termination condition
    epsilon = 1e-6
    
    # initialize the value function
    V_star = np.zeros((len(games), len(upsilons), n_states))
    
    for idx_game, game in enumerate(games):
        if progress:
            print("\rComputation of V_star: game #{:5d}".format(idx_game+1), end="")
        for idx_upsilon, upsilon in enumerate(upsilons):
            
            V = np.zeros(n_states)
            dV = 1.0
            n_iters = 0
            
            # repeat until convergence
            while dV >= epsilon:
                dV = 0.0
                n_iters += 1
                
                for state in range(n_states):
                    prev_V = V[state]

                    # compute risk for all actions
                    risk = np.zeros(n_actions)
                    for action in range(n_actions):
                        risk[action] = get_analytical_risk(upsilon, action, game[:,state,:], V)

                    # update value function
                    V[state] = upsilon[3][state] + upsilon[2]*np.min(risk)

                    # get update difference
                    dV = np.maximum(dV, np.abs(prev_V - V[state]))
                    
                if n_iters >= 150:
                    break
            
            V_star[idx_game, idx_upsilon, :] = V
    
    if progress:
        print(" Done.")
    return V_star

Define the "regret" as

\begin{align*}
\tilde{\Phi}(\pi;G,\ell) &= \sum_{x \in \mathbb{X}} \tilde{\Phi}^{x}(\pi(x);G,\ell),\\
\tilde{\Phi}^{x}(a;G,\ell) &= C_\ell(x) + r_\ell \rho_{\mu_\ell}\Big(V^{*}_{G,\ell}(X^{x,a}_{G})\Big) - V^{*}_{G,\ell}(x).
\end{align*}

It can be viewed as the regret of the policy $\pi$ under game $G$ and risk aversion $(C_{\ell}, \mu_{\ell}, r_{\ell})$.

In [ ]:
def compute_regret(upsilons, game, pi, V_star):
    phis = np.zeros(len(upsilons))
    
    for idx_upsilon, upsilon in enumerate(upsilons):
        # compute regret for all separate states
        phi_x = np.zeros(n_states)
        for state in range(n_states):
            risk = get_analytical_risk(upsilon, pi[state], game[:,state,:], V_star[idx_upsilon,:])
            phi_x[state] = upsilon[3][state] + upsilon[2]*risk - V_star[idx_upsilon,state]

        # compute regret
        phis[idx_upsilon] = np.sum(phi_x)
    
    return phis

Let $\pi^{*,\ell}$ be the optimal actions under the risk aversion $(C_{\ell}, \mu_{\ell}, r_{\ell})$. Define the "power" as
\begin{align*}
\tilde{\Psi}(G;i,j) &= - \sum_{x \in \mathbb{X}} \tilde{\Phi}^{x}(\pi^{*,j}(x);G,i) \tilde{\Phi}^{x}(\pi^{*,i}(x);G,j) \\
&= - \sum_{x \in \mathbb{X}} \bigg( \tilde{\Phi}^{x}(1;G,i) \tilde{\Phi}^{x}(2;G,j) + \tilde{\Phi}^{x}(2;G,i) \tilde{\Phi}^{x}(1;G,j) \bigg).
\end{align*}

In [ ]:
def compute_power(upsilons, games, V_star, progress=False):
    psis = np.zeros((len(games), len(upsilons), len(upsilons)))
    
    # loop for each lambda
    for idx_game, game in enumerate(games):
        if progress:
            print("\rPower computation: game #{:5d}".format(idx_game+1), end="")            
        
        # loop for each criterion
        for idx_upsilon1, upsilon1 in enumerate(upsilons):
            # loop for other criteria
            for idx_upsilon2, upsilon2 in enumerate(upsilons):
                # make sure \psi(\lambda, \mu_i, \mu_i) is -\inf
                if idx_upsilon1 == idx_upsilon2:
                    psis[idx_game, idx_upsilon1, idx_upsilon1] = -np.inf
                else:
                    # get regret for all pairs of actions / upsilons
                    phi_x_a0_u1 = np.zeros(n_states)
                    phi_x_a1_u1 = np.zeros(n_states)
                    phi_x_a0_u2 = np.zeros(n_states)
                    phi_x_a1_u2 = np.zeros(n_states)

                    for state in range(n_states):
                        phi_x_a0_u1[state] = upsilon1[3][state] - V_star[idx_game,idx_upsilon1,state] \
                            + upsilon1[2]*get_analytical_risk(upsilon1, 0, game[:,state,:], V_star[idx_game,idx_upsilon1,:])
                        phi_x_a1_u1[state] = upsilon1[3][state] - V_star[idx_game,idx_upsilon1,state] \
                            + upsilon1[2]*get_analytical_risk(upsilon1, 1, game[:,state,:], V_star[idx_game,idx_upsilon1,:])
                        phi_x_a0_u2[state] = upsilon2[3][state] - V_star[idx_game,idx_upsilon2,state] \
                            + upsilon2[2]*get_analytical_risk(upsilon2, 0, game[:,state,:], V_star[idx_game,idx_upsilon2,:])
                        phi_x_a1_u2[state] = upsilon2[3][state] - V_star[idx_game,idx_upsilon2,state] \
                            + upsilon2[2]*get_analytical_risk(upsilon2, 1, game[:,state,:], V_star[idx_game,idx_upsilon2,:])

                    # compute the power
                    psis[idx_game, idx_upsilon1, idx_upsilon2] = -1 * np.sum(phi_x_a0_u1*phi_x_a1_u2 \
                                                                             + phi_x_a1_u1*phi_x_a0_u2)
    
    if progress:
        print(" Done.")
    return psis

The `select_nextgame` function determines which transition probability matrix to use according to three different methods:

- **"random"** randomly selects a game
- **"largest"** selects the next game according to
\begin{equation*}
\arg\!\min_{G} \tilde{\Psi}(G;\eta,\zeta),
\end{equation*}
where $(\eta,\zeta)$ is the pair of entries with the largest and second largest probabilities assigned by $\mathbb{Q}$.
- **"expected"** selects the next game according to
\begin{equation*}
\arg\!\min_{G} \mathbb{E}_{(\eta,\zeta) \sim \mathbb{Q}|_{\eta \neq \zeta}} \Big[ \tilde{\Psi}(G;\eta,\zeta) \Big].
\end{equation*}

In [ ]:
def select_nextgame(psis, upsilons, gibbs_meas, games, n_sims, method):
    if method=="random":
        next_game = np.random.randint(len(games))
    
    elif method=="largest":
        # obtain the two risk-aversions with the largest Gibbs value
        largest = np.partition(gibbs_meas, -2)[-2:]
        if largest[0] == largest[1]:
            upsilon_largest = np.random.choice(np.where(gibbs_meas == largest[1])[0], size=2)
        else:
            upsilon_largest = np.array([np.where(gibbs_meas == largest[1])[0][0],
                                        np.random.choice(np.where(gibbs_meas == largest[0])[0])])
        
        next_game = np.argmin( psis[ :, upsilon_largest[0], upsilon_largest[1]] )
       
    elif method=="expected":
        # sample two criteria (without replacement) according to the Gibbs measure
        upsilon_rand = np.stack([np.random.choice(len(upsilons),size=2,replace=False,p=gibbs_meas) for i in range(n_sims)])
        
        # compute the expectation of the difference in power
        expected_powers = np.zeros(len(games))
        for idx_game, game in enumerate(games):
            expected_powers[idx_game] = np.mean( psis[ idx_game, upsilon_rand[:,0], upsilon_rand[:,1]] )

        next_game = np.argmin(expected_powers)
        
    else:
        ValueError("Selection method is unknown ('random' or 'largest' or 'expected').")
        
    return next_game, games[next_game, ...]

The `learn` function runs the main learning algorithm.

In [ ]:
def learn(games, # set of games
          V_star, # V^{*} for all games
          psis, # power for all games
          upsilons, # set of criteria for the learner
          upsilon0, # criterion for the expert
          n_rounds, # number of rounds
          select_method, # string for the selection method when designing games at each episode
          lr, # learning rate for the Gibbs measure
          print_progress=None): # number of episodes before printing the progress

    # initialize Gibbs measure
    phis_hat = np.zeros(len(upsilons))
    gibbs_meas = np.exp(phis_hat-np.max(phis_hat)) / np.sum(np.exp(phis_hat-np.max(phis_hat)))
    
    true_gibbs = np.zeros((n_rounds, len(upsilons)))
        
    # algorithm
    for k in range(n_rounds):

        # select the lambda with highest power
        idx_game, game = select_nextgame(psis, upsilons, gibbs_meas, games, n_sims=100, method=select_method)

        # get optimal action from analytical risk
        opt_pi = np.zeros(n_states, dtype=int)
        opt_V_star = compute_Vstar([upsilon0], game[np.newaxis, :], progress=False).squeeze()
        for state in range(n_states):
            all_actions = np.zeros(n_actions)
            for action in range(n_actions):
                all_actions[action] = get_analytical_risk(upsilon0, action, game[:,state,:], opt_V_star)
            opt_pi[state] = np.argmin(all_actions)
        
        # compute the potential/energy for all upsilons
        phis_k = compute_regret(upsilons, game, opt_pi, V_star[idx_game,:,:])
        ###print(np.array_str(phis_k, precision=5, suppress_small=True))
        
        # normalize the potentials
        phis_normalized = phis_k - np.mean(phis_k)

        # update sum of potentials
        phis_hat -= phis_normalized

        # update Gibbs measure (subtracting the maximum for numerical stability)
        gibbs_meas = np.exp(lr*(phis_hat-np.max(phis_hat))) / np.sum(np.exp(lr*(phis_hat-np.max(phis_hat))))
        
        # keep track Gibbs measure for true risk-aversion
        true_gibbs[k,...] = gibbs_meas
        
        # print progress
        if (print_progress is not None) and (k % print_progress == 0 or k == n_rounds - 1):
            print("Episode: {:3d};".format(k+1),
                  "Best criterion: {:2d}".format(np.argmax(gibbs_meas) + 1),
                  "with kappa: {:.2f}".format(upsilons[np.argmax(gibbs_meas)][0]),
                  "  gamma: {:.2f}".format(upsilons[np.argmax(gibbs_meas)][1]),
                  "  discount: {:.2f}".format(upsilons[np.argmax(gibbs_meas)][2]),
                  "  cost function: ", np.array_str(upsilons[np.argmax(gibbs_meas)][3], precision=2))        
            print("Gibbs:", np.array_str(gibbs_meas, precision=2, suppress_small=True))
    
    return true_gibbs

### Initialization of Hyperparameters

Once we perform the training of the $V$, we run the learning algorithm. The expert specifies its criterion, denoted $\Xi_0$, while the learner specifies the set of criteria, the number of games to explore when designing the next game, and the learning rate. Note here that the expert criterion may be a different criterion than the ones in the criterion space.

In [ ]:
"""
EXPERT AND LEARNER PARAMETERS
"""
### set of experiments 1
upsilon0 = (0.4, 0.2, 0.4, np.array([1.0, 0.5, 0.0]))
kappas = np.array([0.2, 0.3, 0.4])
gammas = np.array([0.2, 0.5])
discounts = np.array([0.2, 0.4, 0.6])
unknown_cost = True
costs = np.array([[1.0, 0.5, 0.0],
                [0.5, 1.0, 0.0]])

# ### set of experiments 2
# upsilon0 = (0.25, 0.0, 0.8, np.array([1.0, 0.5, 0.0]))
# kappas = np.array([0.25])
# gammas = np.array([0.0])
# discounts = np.linspace(0.1, 0.9, 21)
# unknown_cost = False
# costs = np.array([[1.0, 0.5, 0.0]])

upsilons = list(itertools.product(kappas, gammas, discounts, costs))

"""
ALGORITHM PARAMETERS
"""
n_games = 500 # number of games
lr = 4 # learning rate

In [ ]:
# validate all parameters
assert isinstance(n_games, int) & (n_games >= 0), "n_games must be a positive integer."
assert np.all((kappas > 0) & (kappas < 1)), "All kappas must be in (0,1)."
assert np.all((gammas >= 0) & (gammas < 1)), "All gammas must be in [0,1)."
assert np.all((discounts >= 0) & (discounts < 1)), "All discount factors must be in [0,1)."
assert len(upsilon0) == 4, "upsilon0 must contain a (kappa, gamma, discount factor, cost function)."
print('*** All parameters are validated ***')

In [ ]:
print("Agent criterion --  kappa: {:.2f}".format(upsilon0[0]),
      "  gamma: {:.2f}".format(upsilon0[1]),
      "  discount: {:.2f}".format(upsilon0[2]),
      "  cost function: ", np.array_str(upsilon0[3], precision=2))
print("--------------")
print('Criterion space (', 'unknown' if unknown_cost else 'known', ' cost):', sep='')
for idx, upsilon in enumerate(upsilons):
    print("Criterion #{:2d}".format(idx+1),
          " --  kappa: {:.2f}  ".format(upsilon[0]), 
          "  gamma: {:.2f}  ".format(upsilon[1]),
          "  discount: {:.2f}  ".format(upsilon[2]),
          "  cost function: ", np.array_str(upsilon[3], precision=2))

### Computation of the power for all games

We first sample many games, and then compute the power of all games with respect to all pairs of risk-aversions. This is independent of the selection method for the next game, which is why we execute this outside the `learn` function.

In [ ]:
np.random.seed(1234)

# generate a set of games (random transition probabilities)
z = np.random.randn(n_games, n_actions, n_states, n_states)
games = np.exp(z) / np.sum(np.exp(z), axis=3, keepdims=True)

# compute V_star for all games
V_star = compute_Vstar(upsilons, games, progress=True)

# compute power for all games
psis = compute_power(upsilons, games, V_star, progress=True)

### Learning algorithm with uniform game design

In this setting, we first validate that the Gibbs measure converges to the dirac delta function when using uniformly randomized games at each episode.

In [ ]:
np.random.seed(1234)
gibbs = \
    learn(games=games,
          V_star=V_star,
          psis=psis,
          upsilons=upsilons,
          upsilon0=upsilon0,
          n_rounds=10_000,
          select_method="random",
          lr=lr,
          print_progress=1_000)

In [ ]:
fig, axes = plt.subplots(figsize=(10,6))

# lines
a = gibbs[len(gibbs)//2,:]
legend = ["_nolegend_" if a[i] < np.partition(a, -4)[-4] \
                   else r"$\Upsilon_{{{:2d}}}$".format(i+1)+\
                          r": $\kappa=$ {:.3f}".format(upsilons[i][0])+ \
                          r"  $\gamma=$ {:.3f}".format(upsilons[i][1])+ \
                          r"  $r=:$ {:.2f}  ".format(upsilon[2]) + \
                          "  cost: "+ np.array_str(upsilons[i][3], precision=2) for i in range(len(a))]

axes.plot(np.arange(len(gibbs)),
          gibbs,
          linewidth=2.0,
          label=legend)

# aesthetics
# axes.set_title('Convergence of the Gibbs measure', fontsize=24)
axes.set_xlabel('Rounds', fontsize=20)
axes.set_ylabel(r"Values of $\mathbb{Q}_{N}$", fontsize=20)
axes.set_xlim(0, len(gibbs))
axes.set_ylim(0, 1)
axes.legend(loc=9, bbox_to_anchor=(0.5,-0.12), fontsize=18)
axes.tick_params(axis='both', labelsize=14)
plt.savefig("figures/uniform-gibbs-infinite.pdf", transparent=False, bbox_inches='tight')
plt.clf()
plt.close()
# plt.show()

### Learning algorithm with minimization of power for largest risk-aversions

In this setting, we verify if the Gibbs measure converges faster when designing the next game according to the power of the largest risk-aversion values.

In [ ]:
np.random.seed(1234)
gibbs = \
    learn(games=games,
          V_star=V_star,
          psis=psis,
          upsilons=upsilons,
          upsilon0=upsilon0,
          n_rounds=500,
          select_method="largest",
          lr=lr,
          print_progress=50)

In [ ]:
fig, axes = plt.subplots(figsize=(10,6))

# lines
a = gibbs[len(gibbs)//2,:]
legend = ["_nolegend_" if a[i] < np.partition(a, -4)[-4] \
                   else r"$\Upsilon_{{{:2d}}}$".format(i+1)+\
                          r": $\kappa=$ {:.3f}".format(upsilons[i][0])+ \
                          r"  $\gamma=$ {:.3f}".format(upsilons[i][1])+ \
                          r"  $r=:$ {:.2f}  ".format(upsilon[2]) + \
                          "  cost: "+ np.array_str(upsilons[i][3], precision=2) for i in range(len(a))]

axes.plot(np.arange(len(gibbs)),
          gibbs,
          linewidth=2.0,
          label=legend)

# aesthetics
# axes.set_title('Convergence of the Gibbs measure', fontsize=24)
axes.set_xlabel('Rounds', fontsize=20)
axes.set_ylabel(r"Values of $\mathbb{Q}_{N}$", fontsize=20)
axes.set_xlim(0, 500)
axes.set_ylim(0, 1)
axes.legend(loc=9, bbox_to_anchor=(0.5,-0.12), fontsize=18)
axes.tick_params(axis='both', labelsize=14)
plt.savefig("figures/largest-gibbs-infinite.pdf", transparent=False, bbox_inches='tight')
plt.clf()
plt.close()
# plt.show()

### Learning algorithm with minimization of expected power

In this setting, we verify if the Gibbs measure converges faster when designing the next game according to the expected power.

In [ ]:
np.random.seed(1234)
gibbs = \
    learn(games=games,
          V_star=V_star,
          psis=psis,
          upsilons=upsilons,
          upsilon0=upsilon0,
          n_rounds=500,
          select_method="expected",
          lr=lr,
          print_progress=50)

In [ ]:
fig, axes = plt.subplots(figsize=(10,6))

# lines
a = gibbs[len(gibbs)//2,:]
legend = ["_nolegend_" if a[i] < np.partition(a, -4)[-4] \
                   else r"$\Upsilon_{{{:2d}}}$".format(i+1)+\
                          r": $\kappa=$ {:.3f}".format(upsilons[i][0])+ \
                          r"  $\gamma=$ {:.3f}".format(upsilons[i][1])+ \
                          r"  $r=:$ {:.2f}  ".format(upsilon[2]) + \
                          "  cost: "+ np.array_str(upsilons[i][3], precision=2) for i in range(len(a))]

axes.plot(np.arange(len(gibbs)),
          gibbs,
          linewidth=2.0,
          label=legend)

# aesthetics
# axes.set_title('Convergence of the Gibbs measure', fontsize=24)
axes.set_xlabel('Rounds', fontsize=20)
axes.set_ylabel(r"Values of $\mathbb{Q}_{N}$", fontsize=20)
axes.set_xlim(0, 500)
axes.set_ylim(0, 1)
axes.legend(loc=9, bbox_to_anchor=(0.5,-0.12), fontsize=18)
axes.tick_params(axis='both', labelsize=14)
plt.savefig("figures/expected-gibbs-infinite.pdf", transparent=False, bbox_inches='tight')
plt.clf()
plt.close()
# plt.show()

### Convergence speed visualizations

In this setting, we perform multiple runs of the learning algorithm for the different game-design rules. This allows an estimation of the distribution for $\mathbb{Q}_{N}(\{0\})$, i.e. the Gibbs measure value for the expert's true risk-aversion, at each round.

In [ ]:
np.random.seed(4321)
n_runs = 25
n_rounds1 = 5000
n_rounds2 = 5000
n_games = 500
lr = 4

idx_upsilon0 = [idx for idx,ii in enumerate(upsilons) if (ii[0] == upsilon0[0]) and \
                                                            (ii[1] == upsilon0[1]) and \
                                                            (ii[2] == upsilon0[2]) and \
                                                            (ii[3] == upsilon0[3]).all()][0]

unif_gibbs = np.zeros((n_runs,n_rounds1,len(upsilons)))
largest_gibbs = np.zeros((n_runs,n_rounds2,len(upsilons)))
expected_gibbs = np.zeros((n_runs,n_rounds2,len(upsilons)))

for run in range(n_runs):
    
    print("\rProgress: run #{:5d} of {:5d}".format(run+1, n_runs), end="")
    
    # generate a set of games (random transition probabilities)
    z = np.random.randn(n_games, n_actions, n_states, n_states)
    games = np.exp(z) / np.sum(np.exp(z), axis=3, keepdims=True)

    # compute V_star for all games
    V_star = compute_Vstar(upsilons, games, progress=False)

    # compute power for all games
    psis = compute_power(upsilons, games, V_star, progress=False)

    unif_gibbs[run,:] = \
        learn(games=games,
              V_star=V_star,
              psis=psis,
              upsilons=upsilons,
              upsilon0=upsilon0,
              n_rounds=n_rounds1,
              select_method="random",
              lr=lr)
    largest_gibbs[run,:] = \
        learn(games=games,
              V_star=V_star,
              psis=psis,
              upsilons=upsilons,
              upsilon0=upsilon0,
              n_rounds=n_rounds2,
              select_method="largest",
              lr=lr)
    expected_gibbs[run,:] = \
        learn(games=games,
              V_star=V_star,
              psis=psis,
              upsilons=upsilons,
              upsilon0=upsilon0,
              n_rounds=n_rounds2,
              select_method="expected",
              lr=lr)

In [ ]:
from matplotlib.ticker import ScalarFormatter

fig, axes = plt.subplots(figsize=(10,6))

largest_upsilon0 = largest_gibbs[...,idx_upsilon0]
expected_upsilon0 = expected_gibbs[...,idx_upsilon0]
unif_upsilon0 = unif_gibbs[...,idx_upsilon0]

# lines
axes.plot(np.arange(n_rounds2),
          np.mean(largest_upsilon0, axis=0),
          label='largest', color='C0', linewidth=1.5)
axes.plot(np.arange(n_rounds2),
          np.mean(expected_upsilon0, axis=0),
          label='expected', color='C1', linewidth=1.5)
axes.plot(np.arange(n_rounds1),
          np.mean(unif_upsilon0, axis=0),
          label='uniform', color='C2', linewidth=1.5)

# shades
axes.fill_between(np.arange(n_rounds2),
                  np.quantile(largest_upsilon0, 0.9, axis=0),
                  np.quantile(largest_upsilon0, 0.1, axis=0),
                  facecolor='C0', alpha=0.3)
axes.fill_between(np.arange(n_rounds2),
                  np.quantile(expected_upsilon0, 0.9, axis=0),
                  np.quantile(expected_upsilon0, 0.1, axis=0),
                  facecolor='C1', alpha=0.3)
axes.fill_between(np.arange(n_rounds1),
                  np.quantile(unif_upsilon0, 0.9, axis=0),
                  np.quantile(unif_upsilon0, 0.1, axis=0),
                  facecolor='C2', alpha=0.3)

# aesthetics
# axes.set_title('Convergence of the Gibbs measure to the true risk-aversion', fontsize = 24)
axes.set_xlabel(r'Rounds (log-scale)', fontsize=20)
axes.set_ylabel(r"Value of $\tilde{\mathbb{Q}}_{N}(\{0\})$", fontsize=20)
axes.legend(loc='upper left', fontsize=18)
axes.set_xlim(1, n_rounds1)
axes.set_xscale('log')
axes.xaxis.set_major_formatter(ScalarFormatter())
axes.set_xticks([1,10,100,1000,5000])
axes.set_ylim(0, 1)
axes.tick_params(axis='both', labelsize=14)
plt.savefig("figures/gibbs-infinite.pdf", transparent=False, bbox_inches='tight')
plt.clf()
plt.close()
# plt.show()

### Learning rate visualizations

In this setting, we perform multiple runs of the learning algorithm for the different game-design rules, but vary the learning rate to understand the behavior of this hyperparameter.

In [ ]:
np.random.seed(4321)
n_runs = 25
n_rounds = 50
n_games = 500
lrs = np.linspace(0.1, 10.0, 25)

idx_upsilon0 = [idx for idx,ii in enumerate(upsilons) if (ii[0] == upsilon0[0]) and \
                                                            (ii[1] == upsilon0[1]) and \
                                                            (ii[2] == upsilon0[2]) and \
                                                            (ii[3] == upsilon0[3]).all()][0]

unif_gibbs = np.zeros((len(lrs),n_runs,n_rounds,len(upsilons)))
largest_gibbs = np.zeros((len(lrs),n_runs,n_rounds,len(upsilons)))
expected_gibbs = np.zeros((len(lrs),n_runs,n_rounds,len(upsilons)))

for run in range(n_runs):
    
    print("\rProgress: run #{:5d} of {:5d}".format(run+1, n_runs), end="")
    
    # generate a set of games (random transition probabilities)
    z = np.random.randn(n_games, n_actions, n_states, n_states)
    games = np.exp(z) / np.sum(np.exp(z), axis=3, keepdims=True)

    # compute V_star for all games
    V_star = compute_Vstar(upsilons, games, progress=False)

    # compute power for all games
    psis = compute_power(upsilons, games, V_star, progress=False)
    
    for idx_lr, lr in enumerate(lrs):

        unif_gibbs[idx_lr,run,...] = \
            learn(games=games,
                  V_star=V_star,
                  psis=psis,
                  upsilons=upsilons,
                  upsilon0=upsilon0,
                  n_rounds=n_rounds,
                  select_method="random",
                  lr=lr)
        largest_gibbs[idx_lr,run,...] = \
            learn(games=games,
                  V_star=V_star,
                  psis=psis,
                  upsilons=upsilons,
                  upsilon0=upsilon0,
                  n_rounds=n_rounds,
                  select_method="largest",
                  lr=lr)
        expected_gibbs[idx_lr,run,...] = \
            learn(games=games,
                  V_star=V_star,
                  psis=psis,
                  upsilons=upsilons,
                  upsilon0=upsilon0,
                  n_rounds=n_rounds,
                  select_method="expected",
                  lr=lr)

print(" Done.")

In [ ]:
fig, axes = plt.subplots(figsize=(10,6))

largest_upsilon0 = largest_gibbs[...,-1,idx_upsilon0]
expected_upsilon0 = expected_gibbs[...,-1,idx_upsilon0]
unif_upsilon0 = unif_gibbs[...,-1,idx_upsilon0]

# lines
axes.plot(lrs,
          np.mean(largest_upsilon0, axis=1),
          label='largest', color='C0', linewidth=1.5)
axes.plot(lrs,
          np.mean(expected_upsilon0, axis=1),
          label='expected', color='C1', linewidth=1.5)
axes.plot(lrs,
          np.mean(unif_upsilon0, axis=1),
          label='uniform', color='C2', linewidth=1.5)

# shades
axes.fill_between(lrs,
                  np.quantile(largest_upsilon0, 0.9, axis=1),
                  np.quantile(largest_upsilon0, 0.1, axis=1),
                  facecolor='C0', alpha=0.3)
axes.fill_between(lrs,
                  np.quantile(expected_upsilon0, 0.9, axis=1),
                  np.quantile(expected_upsilon0, 0.1, axis=1),
                  facecolor='C1', alpha=0.3)
axes.fill_between(lrs,
                  np.quantile(unif_upsilon0, 0.9, axis=1),
                  np.quantile(unif_upsilon0, 0.1, axis=1),
                  facecolor='C2', alpha=0.3)

# aesthetics
# axes.set_title('Convergence of the Gibbs measure to the true risk-aversion\n after a given number of rounds', fontsize = 24)
axes.set_xlabel('Learning rate', fontsize=20)
axes.set_ylabel(r"Value of $\tilde{\mathbb{Q}}_{N}(\{0\})$", fontsize=20)
axes.legend(loc='upper left', fontsize=18)
axes.set_xlim(np.min(lrs), np.max(lrs))
axes.set_ylim(0, 1)
axes.tick_params(axis='both', labelsize=14)
plt.savefig("figures/all-lrs-infinite.pdf", transparent=False, bbox_inches='tight')
plt.clf()
plt.close()
# plt.show()